<a href="https://colab.research.google.com/github/HanSong19/Hugging-Face/blob/main/5.3%20Big%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Big Data

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00


In [78]:
!pip install zstandard

In [10]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2023-08-09 08:17:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [           <=>      ]  41.00M  15.6MB/s    in 2.6s    

2023-08-09 08:17:57 (15.6 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [11]:
from datasets import load_dataset

data_files = "drugsComTrain_raw.tsv"
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
drug_dataset['train'][0]

{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'Left Ventricular Dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27}

In [19]:
!pip install psutil
import psutil
# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 253.08 MB


In [31]:
print(f"Number of files in dataset : {drug_dataset['train'].dataset_size}")
size_gb = drug_dataset['train'].dataset_size / (1024**3)
print(f"Dataset size (cache file) : {size_gb:.2f} GB")

Number of files in dataset : 87127744
Dataset size (cache file) : 0.08 GB


In [35]:
import timeit

code_snippet = """batch_size = 1000

for idx in range(0, len(drug_dataset['train']), batch_size):
    _ = drug_dataset['train'][idx:idx + batch_size]
"""

time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
print(
    f"Iterated over {len(drug_dataset)} examples (about {size_gb:.1f} GB) in "
    f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s"
)

Iterated over 1 examples (about 0.1 GB) in 0.9s, i.e. 0.092 GB/s


##stream data

In [39]:
drug_data_stream = load_dataset("csv", data_files=data_files, streaming=True, delimiter = '\t' )
next(iter(drug_data_stream['train']))

{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'Left Ventricular Dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27}

In [41]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_drug_data =drug_data_stream.map(lambda x: tokenizer(x["review"]))
next(iter(tokenized_drug_data['train']))

{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'Left Ventricular Dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'input_ids': [101,
  1000,
  2009,
  2038,
  2053,
  2217,
  3466,
  1010,
  1045,
  2202,
  2009,
  1999,
  5257,
  1997,
  2011,
  16033,
  10415,
  1019,
  11460,
  1998,
  3869,
  3514,
  1000,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [44]:
shuffle_drug_dataset = drug_data_stream.shuffle(buffer_size=100, seed=10)
next(iter(shuffle_drug_dataset['train']))

{'Unnamed: 0': 9116,
 'drugName': 'Lo Loestrin Fe',
 'condition': 'Birth Control',
 'review': '"I am always bleeding between periods, I&rsquo;ve also gained 15 pounds and on a skinny girl it&rsquo;s very noticeable! My acne hasn&rsquo;t got worse or better. And now I&rsquo;m not getting my period when I should be. I am experiencing depression where I don&rsquo;t want to be around anyone I don&rsquo;t want to eat because I&rsquo;m gaining so much weight I&rsquo;m stuck in my head and lashing out at people. I&rsquo;m not my normal self. I wish I found a pill that rids my acne and doesn&rsquo;t affect my weight or sex drive.."',
 'rating': 2.0,
 'date': 'October 26, 2017',
 'usefulCount': 5}

In [50]:
drug_dataset_head = drug_data_stream['train'].take(5)
list(drug_dataset_head)

[{'Unnamed: 0': 206461,
  'drugName': 'Valsartan',
  'condition': 'Left Ventricular Dysfunction',
  'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
  'rating': 9.0,
  'date': 'May 20, 2012',
  'usefulCount': 27},
 {'Unnamed: 0': 95260,
  'drugName': 'Guanfacine',
  'condition': 'ADHD',
  'review': '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is bett

In [60]:
train_drug_dataset = shuffle_drug_dataset['train'].skip(1000)

test_drug_dataset = shuffle_drug_dataset['train'].take(1000)

{'Unnamed: 0': 222679, 'drugName': 'Topamax', 'condition': 'Migraine Prevention', 'review': '"I&#039;ve taking topamax for three years.  It helped but I started getting eye pain behind my left eye.  My eye would throb like a bird was pecking at it. I went to several doctors about it and U of L dr. figured it out...it was topamax causing the problem.  I&#039;m  off of the meds.and having very little pain behind my eye."', 'rating': 5.0, 'date': 'July 27, 2016', 'usefulCount': 23}


In [79]:

base_url = "https://the-eye.eu/public/AI/pile/"

data_files = {
    "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(30)],
    "test": base_url + "val.jsonl.zst",
    "validation": base_url+  "test.jsonl.zst",
}

pile_data = load_dataset("json", data_files=data_files, streaming=True)
next(iter(pile_data["train"]))

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

ValueError: ignored